In [0]:
!pip install vecstack

In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


In [0]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/CIS508/Asg-3 Insurance Fraud Smote Stacking/IF_Train.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508/Asg-3 Insurance Fraud Smote Stacking/IF_Test.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

In [0]:
categoricalFeatures = ["MONTH",
"DAYOFWEEK",
"MAKE",
"ACCIDENTAREA",
"DAYOFWEEKCLAIMED",
"MONTHCLAIMED",
"SEX",
"MARITALSTATUS",
"FAULT",
"POLICYTYPE",
"VEHICLECATEGORY",
"VEHICLEPRICE",
"DAYS_POLICY_ACCIDENT",
"DAYS_POLICY_CLAIM",
"PASTNUMBEROFCLAIMS",
"AGEOFVEHICLE",
"AGEOFPOLICYHOLDER",
"POLICEREPORTFILED",
"WITNESSPRESENT",
"AGENTTYPE",
"NUMBEROFSUPPLIMENTS",
"ADDRESSCHANGE_CLAIM",
"NUMBEROFCARS",
"YEAR",
"BASEPOLICY"]


#Copy Train data excluding target
trainData_Copy = train_data.iloc[:, :-1].copy()
testData_Copy = test_data.iloc[:, :-1].copy()

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData_Copy,testData_Copy], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)
#print(combined_Data['FRAUDFOUND'])

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)
y_train=train_data["FRAUDFOUND"]
y_test=test_data["FRAUDFOUND"]

train_data["FRAUDFOUND"]=train_data["FRAUDFOUND"].map({"Yes":1,"No":0})
test_data["FRAUDFOUND"]=test_data["FRAUDFOUND"].map({"Yes":1,"No":0})

#Select just Target Column
y_train = train_data.iloc[:, -1]
y_test = test_data.iloc[:, -1]

print(X_train.shape)
print(X_test.head()) 

print(y_train.shape)
print(y_test.head()) 

In [0]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)
print("accuracy Score (training) for Decision TreE:{0:6f}".format(clf.score(X_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))


In [0]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())


In [0]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))




In [0]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


In [0]:
#Gradient Boosting ============================================================
search_grid={'n_estimators':[5,10,20, 30, 50],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_test)
print("accuracy Score (training) for Boosting:{0:6f}".format(abc.score(X_test,y_test)))
print("Confusion Matrix for boosting:")
print(confusion_matrix(y_test,abc_predict))

#Randomized Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

In [0]:
#Construct Gradient Boosting Trees using the best parameters
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,y_train)
abc_predict = abc.predict(X_test)
print("accuracy Score (training) after hypertuning for Boosting:{0:6f}".format(abc.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Boosting:")
print(confusion_matrix(y_test,abc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,abc_predict))

#run cross-validation on best parameters, get auc score
abc_cv_score = cross_val_score(abc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())


In [0]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))


In [0]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingClassifier(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


In [0]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))
print("Confusion Matrix after STACKING for Boosting:")
print(confusion_matrix(y_test,y_pred))
print("=== Classification Report ===")
print(classification_report(y_test,y_pred))


In [0]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict_proba(S_Test))

pred_Probability.head()
